In [1]:
import json
import os.path
import pandas as pd
import numpy as np
from preprocessing.logParser import openLogs

In [2]:
BASE_DIR = os.path.abspath('test_statistics.ipynb')
BASE_DIR = os.path.dirname(BASE_DIR)
BASE_DIR = BASE_DIR + '/spbu_ACADRU_spring_2018-TL'
cursor = openLogs(BASE_DIR)

In [33]:
#and questions NOT LIKE '__' means that we needn't questions like {}
#and attempts < 4 means that we use only three first attempts, other attempts are not interesting
request = """
        select course_name,
        attempts, questions,
        page, grade, max_grade
        from problem_check
        where course_name = "course-v1:spbu+ACADRU+spring_2018"
        and questions NOT LIKE '__'
        and attempts < 4
        """.format("https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/244a81ebf8a64b609cd8587bbe18edc5/")
cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)

### модуль тестов, пункт 1 a: (средний результат за тест и медиана)

In [62]:
avg=df.groupby(['course_name','page','attempts','max_grade'])['grade'].agg([np.mean, np.median, np.size])

In [63]:
avg=avg.reset_index()

In [66]:
print(avg.to_json(orient='records'))
#or table?

[{"course_name":"course-v1:spbu+ACADRU+spring_2018","page":"https:\/\/courses.openedu.ru\/courses\/course-v1:spbu+ACADRU+spring_2018\/xblock\/block-v1:spbu+ACADRU+spring_2018+type@problem+block@062bec6c9167fb11f29c\/handler\/xmodule_handler\/problem_check","attempts":1,"max_grade":1.0,"mean":0.6666666667,"median":1.0,"size":3.0},{"course_name":"course-v1:spbu+ACADRU+spring_2018","page":"https:\/\/courses.openedu.ru\/courses\/course-v1:spbu+ACADRU+spring_2018\/xblock\/block-v1:spbu+ACADRU+spring_2018+type@problem+block@08944f89f5a69eb0f3d9\/handler\/xmodule_handler\/problem_check","attempts":1,"max_grade":1.0,"mean":1.0,"median":1.0,"size":2.0},{"course_name":"course-v1:spbu+ACADRU+spring_2018","page":"https:\/\/courses.openedu.ru\/courses\/course-v1:spbu+ACADRU+spring_2018\/xblock\/block-v1:spbu+ACADRU+spring_2018+type@problem+block@0c2e896b3e86420d854609b41ffb7229\/handler\/xmodule_handler\/problem_check","attempts":1,"max_grade":2.0,"mean":0.0,"median":0.0,"size":1.0},{"course_name":

### модуль тестов, пункт 1 с: (процент правильных ответов на вопрос)

In [79]:
#parsing of questions string in two columns: question and result
listForQuestions=pd.DataFrame(columns=('course_name','page','question','attempts','result'))

for rowCounter in range(df.shape[0]):
    currentAttempt=df.attempts[rowCounter]
    currentPage=df.page[rowCounter]
    currentCourse=df.course_name[rowCounter]
    row=(df.questions[rowCounter][1:-1]).split(',')
    for cellRow in row:
        tmprow=cellRow.split(':')
        if tmprow[1]=='True':
            listForQuestions=listForQuestions.append({'course_name':currentCourse,'page':currentPage,'question':tmprow[0],'attempts':currentAttempt,'result':1}, ignore_index=True)
        else:
            listForQuestions=listForQuestions.append({'course_name':currentCourse,'page':currentPage,'question':tmprow[0],'attempts':currentAttempt,'result':0}, ignore_index=True)


In [80]:
listForQuestions['question'] = listForQuestions['question'].map(str.strip)
listForQuestions=listForQuestions.sort_values(by=['question','attempts'])

In [81]:
perc=listForQuestions.groupby(['course_name','page', 'attempts','question'])['result'].agg([np.sum , np.size])

In [82]:
perc=100*perc['sum']/perc['size']

In [83]:
perc=perc.reset_index()


In [86]:
print(perc.shape)

(419, 5)


In [84]:
perc = perc.rename(columns={0: 'percentage_of_right_answers'})
avg = avg.rename(columns={'size': 'number_of_solutions'})

In [88]:
 res = perc.groupby(perc.columns.drop('question').tolist(), as_index=False).agg(list)

In [89]:
print(res)

                           course_name  \
0    course-v1:spbu+ACADRU+spring_2018   
1    course-v1:spbu+ACADRU+spring_2018   
2    course-v1:spbu+ACADRU+spring_2018   
3    course-v1:spbu+ACADRU+spring_2018   
4    course-v1:spbu+ACADRU+spring_2018   
5    course-v1:spbu+ACADRU+spring_2018   
6    course-v1:spbu+ACADRU+spring_2018   
7    course-v1:spbu+ACADRU+spring_2018   
8    course-v1:spbu+ACADRU+spring_2018   
9    course-v1:spbu+ACADRU+spring_2018   
10   course-v1:spbu+ACADRU+spring_2018   
11   course-v1:spbu+ACADRU+spring_2018   
12   course-v1:spbu+ACADRU+spring_2018   
13   course-v1:spbu+ACADRU+spring_2018   
14   course-v1:spbu+ACADRU+spring_2018   
15   course-v1:spbu+ACADRU+spring_2018   
16   course-v1:spbu+ACADRU+spring_2018   
17   course-v1:spbu+ACADRU+spring_2018   
18   course-v1:spbu+ACADRU+spring_2018   
19   course-v1:spbu+ACADRU+spring_2018   
20   course-v1:spbu+ACADRU+spring_2018   
21   course-v1:spbu+ACADRU+spring_2018   
22   course-v1:spbu+ACADRU+spring_

In [85]:
pd.merge(avg, perc, on=['course_name','page','attempts'], how='outer')

,course_name,page,attempts,max_grade,mean,median,number_of_solutions,question,percentage_of_right_answers
0,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,1.0,0.666667,1.0,3.0,"""062bec6c9167fb11f29c_2_1""",66.666667
1,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,1.0,1.000000,1.0,2.0,"""08944f89f5a69eb0f3d9_2_1""",100.000000
2,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,2.0,0.000000,0.0,1.0,"""0c2e896b3e86420d854609b41ffb7229_2_1""",0.000000
3,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,2.0,0.000000,0.0,1.0,"""0c2e896b3e86420d854609b41ffb7229_3_1""",0.000000
4,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,2,2.0,0.000000,0.0,1.0,"""0c2e896b3e86420d854609b41ffb7229_2_1""",0.000000
5,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,2,2.0,0.000000,0.0,1.0,"""0c2e896b3e86420d854609b41ffb7229_3_1""",0.000000
6,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,14.0,9.739130,11.0,23.0,"""0cb4a601e614492a8c2df8a7d024a012_10_1""",65.217391
7,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,14.0,9.739130,11.0,23.0,"""0cb4a601e614492a8c2df8a7d024a012_11_1""",78.260870
8,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,14.0,9.739130,11.0,23.0,"""0cb4a601e614492a8c2df8a7d024a012_12_1""",86.956522
9,course-v1:spbu+ACADRU+spring_2018,https://courses.openedu.ru/courses/course-v1:s...,1,14.0,9.739130,11.0,23.0,"""0cb4a601e614492a8c2df8a7d024a012_13_1""",86.956522
